<a href="https://colab.research.google.com/github/PeterGuk540/DataProjects_AojieJu/blob/main/InitialARV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -y
!apt-get install -y curl unzip xvfb
!wget -O /tmp/chrome.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i /tmp/chrome.deb || apt-get -f install -y
!google-chrome --version



In [ ]:
import os

In [ ]:
%%writefile config.json
{
  "year_built": {
    "offset": 5
  },
  "bedrooms": {
    "min_offset": 0,
    "max_offset": 0
  },
  "bathrooms": {
    "min_offset": 0,
    "max_offset": 0
  },
  "sqft": {
    "offset": 100
  },
  "acres": {
    "offset": 10
  }
}

In [ ]:
import json

with open("config.json", "r") as f:
    config = json.load(f)

# To see the offset for year_built:
year_offset = config["year_built"]["offset"]
print("Year Built Offset:", year_offset)

In [ ]:
# -----------------------------------------------
# Install dependencies (run once)
# !pip install selenium ipywidgets fpdf pillow opencv-python-headless

# -----------------------------------------------
# Install dependencies
!pip install selenium
!pip install ipywidgets
!pip install fpdf

import json, time, os, traceback, requests
import ipywidgets as widgets
from IPython.display import display, clear_output
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from fpdf import FPDF
import cv2, numpy as np
from PIL import Image



# -------------------------------------------------------------------
# PAGE 1: Collect location info (ONLY these three fields)
address_input = widgets.Text(
    value='',
    placeholder='e.g. 157 Meadowlark Cir',
    description='Address:')
state_input = widgets.Dropdown(
    options=['TX','CA','NY','FL','IL'],
    value='TX',
    description='State:')
zip_input = widgets.Text(
    value='',
    placeholder='5‑digit ZIP',
    description='ZIP code:')
go_button = widgets.Button(description="Access Your Report")
page1 = widgets.VBox([address_input, state_input, zip_input, go_button])
display(page1)

def on_go(_):
    clear_output()
    addr = address_input.value.strip()
    st   = state_input.value
    zp   = zip_input.value.strip()
    if not addr or len(zp)!=5 or not zp.isdigit():
        print("🚨 Please enter a valid address and 5‑digit ZIP.")
        display(page1)
        return

    # … then your entire run_search / on_go logic goes here unchanged,
    # parsing addr into street_num, street_name, street_type,
    # logging in, filling the advanced search, extracting the details,
    # re-searching with the ranges, grabbing photos, building the PDF, etc.



    # parse street number, name, type
    parts = addr.split()
    street_num  = parts[0]
    street_type = parts[-1]
    street_name = " ".join(parts[1:-1])

    # 1) login & go to advanced search
    chrome_opts = Options()
    for arg in ["--headless","--no-sandbox","--disable-dev-shm-usage","--disable-gpu"]:
        chrome_opts.add_argument(arg)
    chrome_opts.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(options=chrome_opts)
    wait = WebDriverWait(driver,15)
    try:
        driver.get("https://austin.clareity.net/clp/login")
        wait.until(EC.presence_of_element_located((By.NAME,"username")))
        driver.find_element(By.NAME,"username").send_keys("831268")
        driver.find_element(By.NAME,"password").send_keys("CheckARV,2025")
        driver.find_element(By.ID,"loginbtn").click()
        wait.until(EC.url_changes("https://austin.clareity.net/clp/login"))

        # go to Advanced (home) search
        driver.get("https://matrix.abor.com/Matrix/Home?c=H4sIAAAAAAAEAItWMjc0NlHSySvNyRklyCeUDm*2jFE6PAk9JGMBdsmKSmEBAAA)&f=")
        time.sleep(2)

        # fill Street #
        min_num = int(street_num)
        max_num = min_num
        driver.find_element(By.ID,"Min_Fm67_Ctrl237_TB").send_keys(str(min_num))
        driver.find_element(By.ID,"Max_Fm67_Ctrl237_TB").send_keys(str(max_num))

        # fill Street Name
        driver.find_element(By.ID,"Fm67_Ctrl238_TextBox").send_keys(street_name)

        # Street Type: open modal & check exactly one
        driver.find_element(By.ID,"dropdown_Fm67_Ctrl239_LB").click()
        # wait and find the checkbox with data-mtrx-item-text=street_type
        xpath = f"//input[@type='checkbox' and @data-mtrx-item-text='{street_type}']"
        wait.until(EC.element_to_be_clickable((By.XPATH,xpath))).click()
        # apply
        driver.find_element(By.XPATH,"//input[@value='Apply']").click()

        # fill ZIP
        driver.find_element(By.ID,"Fm67_Ctrl60_TextBox").send_keys(zp)

         # ─── SCREENSHOT BEFORE CLICK ───────────────────────────────────────────────
        driver.save_screenshot("before_home_search.png")

        # locate & click the Home-page Search button
        search_btn = wait.until(EC.element_to_be_clickable((
            By.CSS_SELECTOR,
            "button[data-mtx-track='Homepage - Search - Search Click']"
        )))
        search_btn.click()

                # ─── SCREENSHOT AFTER CLICK ────────────────────────────────────────────────
        time.sleep(1)
        # ─── WAIT FOR THE LISTING ID LINK TO APPEAR ─────────────────────────────
        result_link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,
        "a[href*='/Matrix/Display/GetDisplay']")))

        driver.save_screenshot("before_first_listing.png")

        # click through
        result_link.click()

        time.sleep(1)
        driver.save_screenshot("after_first_listing.png")


        # ─── scrape the detail fields ───────────────────────────────────────────

        def _get(field, default=""):
            """
            Finds the <div> whose text is exactly `field` and has class 'border-bottom',
            then returns the text of its very next sibling <div> that has class 'fw-bold'.
            """
            try:
                xpath = (
                    f"//div[normalize-space()='{field}' and contains(@class,'border-bottom')]"
                    "/following-sibling::div[contains(@class,'fw-bold')][1]"
                )
                ele = driver.find_element(By.XPATH, xpath)
                return ele.text.strip().replace(",", "")
            except Exception:
                return default

        def _get_int(field, default=0):
            txt = _get(field, "")
            try:
                return int(txt)
            except:
                return default

        def _get_float(field, default=0.0):
            txt = _get(field, "")
            try:
                return float(txt)
            except:
                return default

        # now call with the exact labels you see in the DOM:
        beds      = _get_int("# Beds")
        baths     = _get_int("# Baths")
        sqft      = _get_int("SqFt")
        acres     = _get_float("Acres")         # matches <div>…Acres</div>
        yearbuilt = _get_int("Year Built")      # matches <div>…Year Built</div>


        print("Extracted:", beds, baths, sqft, acres, yearbuilt)

        # now compute your ranges:
        yr_off   = config["year_built"]["offset"]
        bd_min   = beds + config["bedrooms"]["min_offset"]
        bd_max   = beds + config["bedrooms"]["max_offset"]
        bt_min   = baths + config["bathrooms"]["min_offset"]
        bt_max   = baths + config["bathrooms"]["max_offset"]
        sqft_min = max(0, sqft - config["sqft"]["offset"])
        sqft_max = sqft + config["sqft"]["offset"]
        acr_min  = max(0, acres - config["acres"]["offset"])
        acr_max  = acres + config["acres"]["offset"]



        driver.get("https://matrix.abor.com/Matrix/Search/Residential/Residential")
        time.sleep(5)

        active_checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, "//label[contains(.,'Active')]/preceding-sibling::input[@type='checkbox']")))
        if not active_checkbox.is_selected():
            active_checkbox.click()
        closed_checkbox = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//label[contains(.,'Closed')]/preceding-sibling::input[@type='checkbox']")))
        if not closed_checkbox.is_selected():
            closed_checkbox.click()


        # (b) Closed range = 0-360
        closed_input = driver.find_element(By.XPATH, "//label[normalize-space()='Closed']/following::input[@class='textbox'][1]")
        closed_input.clear()
        closed_input.send_keys("0-360")

        driver.find_element(By.CLASS_NAME, "mapSearchDistance").send_keys("1")
        driver.find_element(By.ID, "Fm23_Ctrl19_TB").send_keys(addr)



        time.sleep(2)
        # fill the numeric filters:
        driver.find_element(By.ID,"Fm23_Ctrl55_TB").send_keys(f"{bd_min}-{bd_max}")
        driver.find_element(By.ID,"Fm23_Ctrl56_TB").send_keys(f"{bt_min}-{bt_max}")
        driver.find_element(By.ID,"Fm23_Ctrl59_TB").send_keys(f"{sqft_min}-{sqft_max}")
        driver.find_element(By.ID,"Fm23_Ctrl125_TB").send_keys(f"{acr_min}-{acr_max}")
        driver.find_element(By.ID,"Fm23_Ctrl124_TB").send_keys(f"{yearbuilt-yr_off}-{yearbuilt+yr_off}")

        # Updated robust Results button click
        driver.save_screenshot("before_results_wait.png")
        print("Waiting for the Results button to be clickable...")

        results_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "m_ucSearchButtons_m_lbSearch")))

        driver.execute_script("arguments[0].click();", results_button)
        time.sleep(5)

        driver.save_screenshot("after_click_results.png")
        print("Waiting for the checkboxes to load...")

        # Assuming the search results page has already loaded.
                # ─── click into the very first listing ─────────────────────────────────
        first_listing_anchor = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR,
                "a[data-mtx-track='Results - In-Display Full Link Click']"))
        )
        first_listing_anchor.click()

        # ─── define the price‐scraper helper ────────────────────────────────────
        def _get_price(driver, default="N/A"):
            # 1) Ensure the Details tab is active
            try:
                driver.find_element(By.LINK_TEXT, "Listing").click()
            except:
                pass
            # 2) Wait for and grab the List Price
            xpath = ("//td[span[@class='label' and normalize-space(text())='List Price:']]"
                     "/following-sibling::td[1]//span[@class='field']")
            try:
                el = WebDriverWait(driver, 5).until(
                    EC.visibility_of_element_located((By.XPATH, xpath))
                )
                return el.text.strip()
            except:
                return default

        # ─── helper to process the current listing ──────────────────────────────
        def process_current_listing(driver, wait):
            # 1) Extract price
            price = _get_price(driver)

            # 2) Click Photos & collect images
            try:
                photos_btn = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "Photos")))
                photos_btn.click()
            except TimeoutException:
                pass
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "img.IV_Image.img-responsive")))
            img_urls = [img.get_attribute('src')
                        for img in driver.find_elements(By.CSS_SELECTOR, "img.IV_Image.img-responsive")]

            # 3) Switch back to Listing tab
            try:
                listing_btn = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "Listing")))
                listing_btn.click()
            except TimeoutException:
                pass

            return price, img_urls

        # ─── loop through the first N listings ─────────────────────────────────
        listing_images = {}
        max_listings = 10

        for idx in range(1, max_listings + 1):
            print(f"\nProcessing listing {idx}…")
            price, photos = process_current_listing(driver, wait)
            print(f"Listing {idx} Price: {price}, Photos: {len(photos)} imgs")

            listing_images[f"Listing_{idx}"] = {
                "price": price,
                "photos": photos
            }

            # if reached last, break
            if idx == max_listings:
                break

            # click Next to go to the next listing
            try:
              next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "m_DisplayCore_dpy2")))
              driver.execute_script("arguments[0].click();", next_button)
              print("Clicked 'Next' button to advance.")
            except TimeoutException:
              print("No 'Next' button found.")
              break

            time.sleep(2)

    finally:
        driver.quit()

    # ─── PDF Generation (unchanged) ───────────────────────────────────────────
    if listing_images:
        pdf = FPDF('P', 'mm', 'Letter')
        pdf.set_auto_page_break(auto=True, margin=10)
        pdf.set_font("Helvetica", 'B', 14)

        images_per_page = 12
        img_width = (pdf.w - 40) / 3
        img_height = img_width * 0.75

        for lid, listing_data in listing_images.items():
            urls = listing_data["photos"]
            price = listing_data["price"]

            for i, url in enumerate(urls):
                if i % images_per_page == 0:
                    pdf.add_page()
                    pdf.cell(0, 10, f"Listing ID: {lid}", ln=True, align='C')
                    pdf.cell(0, 10, f"Price: {price}", ln=True, align='C')
                    pdf.ln(5)
                    y_offset = pdf.get_y()

                col = (i % images_per_page) % 3
                row = (i % images_per_page) // 3
                x = 10 + col * (img_width + 5)
                y = y_offset + row * (img_height + 5)

                try:
                    resp = requests.get(url)
                    fn = f"{lid}_{i}.jpg"
                    with open(fn, 'wb') as f:
                        f.write(resp.content)
                    pdf.image(fn, x=x, y=y, w=img_width, h=img_height)
                    os.remove(fn)
                except Exception as e:
                    print(f"Error fetching image {url}: {e}")
                    continue

        pdf.output("Listing_Photos_Report.pdf")
        print("PDF generated successfully: Listing_Photos_Report.pdf")
    else:
        print("No images found; PDF not generated.")

go_button.on_click(on_go)
